# **Prerequisites**
First, install and import the libraries that are required for the preprocessing process.


In [ ]:
!pip install datasets
!pip install pandas
!pip install numpy
!pip install hazm
!pip install num2fawords
!pip install tqdm
!pip install sklearn


In [2]:
from datasets import load_dataset

import pandas as pd
import numpy as np

import hazm
from num2fawords import words, ordinal_words
from tqdm import tqdm

from sklearn.model_selection import train_test_split

import os
import string
import six
import re
import glob

# **Preparing Dataset**
In this section, I used my own dataset, you can use the [Mozilla Common Voice dataset](https://commonvoice.mozilla.org/en/datasets) for free, just note that do not call this dataset through the `load_dataset` function and manually download only the **validation** split and split test and train by yourself. as done below

In [ ]:
data_path = '/home/sadeghi/Notebooks/final_data_for_training_wav2vec2_microphony2.txt'
validation = pd.read_csv(data_path)
validation.shape


(923070, 2)

In [ ]:
validation.head()

,path,sentence
0,/mnt/mehrabikia/2TB/alldata/NevisaLive/0007-m/...,ارزش مادی ارزش مادی و اقتصادی SIL
1,/mnt/mehrabikia/2TB/alldata/NevisaLive/0015--/...,میلیارد نفر از مردم جهان SIL
2,/mnt/mehrabikia/2TB/alldata/NevisaLive/0028--/...,اولین نرم‌افزار تشخیص گفتار زبان فارسی SIL
3,/mnt/mehrabikia/2TB/alldata/NevisaLive/0028--/...,اولین نرم‌افزار دستور زبان گفتار فارسی SIL
4,/mnt/mehrabikia/2TB/alldata/NevisaLive/0028--/...,اولین نرم‌افزار گفتار به گفتار مترجم SIL


In [ ]:
#split validation dataset to train and test
import os
import csv
import numpy as np

def split_data(data, test_percent = 0.01):
	test_len = int(len(data)*test_percent)

	data = np.array(data, dtype=object)
	np.random.shuffle(data)
	data = data.tolist()

	test_data, train_data = data[:test_len], data[test_len:]

	return test_data, train_data

def read_csv(path):
	header = []
	data = []
	with open(path, encoding="utf8") as file:
	    csvreader = csv.reader(file)
	    header = next(csvreader)
	    for row in csvreader:
	    	data.append(row)
	return header, data

def write_csv(path, header, data):
	with open(path, 'w', newline="", encoding="utf8") as file:
	    csvwriter = csv.writer(file)
	    csvwriter.writerow(header)
	    csvwriter.writerows(data)

if __name__ == "__main__":
	header, data = read_csv("/home/sadeghi/Notebooks/final_data_for_training_wav2vec2_microphony2.txt")
	# print(header)
	# print(data[0])

	test_data, train_data = split_data(data)
	print(len(data), len(test_data), len(train_data))

	write_csv('/home/sadeghi/train_data_finall.csv', header, train_data)
	write_csv('/home/sadeghi/test_data_finall.csv', header, test_data)


923070 9230 913840


Now load manually splited data with `load_dataset` function.

In [ ]:
common_voice_train = load_dataset("csv", data_files={"train": "/home/sadeghi/train_data_finall.csv"}, delimiter=",")["train"]
common_voice_test = load_dataset("csv", data_files={"test": '/home/sadeghi/test_data_finall.csv'}, delimiter=",")["test"]
print(common_voice_train)
print(common_voice_test)

Using custom data configuration default-bf07e7c032c09f7c


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /home/sadeghi/.cache/huggingface/datasets/csv/default-bf07e7c032c09f7c/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /home/sadeghi/.cache/huggingface/datasets/csv/default-5a55df3b21dd364a/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['path', 'sentence'],
    num_rows: 913840
})
Dataset({
    features: ['path', 'sentence'],
    num_rows: 9230
})


In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(common_voice_train)

,path,sentence
0,/mnt/mehrabikia/2TB/alldata/FarsSpon/FarsSpon_Retrying/287220537/287220537-1994-1.wav,فکر کنم هندوانه و خامه تازه زیاد با هم جور نیستن
1,/mnt/mehrabikia/2TB/alldata/DeepMine/wav/trans_FAR_ASR/R001783/S028219/P095121.wav,خرید برنج برای سازمان مستقل تدارکات کارمندان کشوری در شمال نزدیک به اتمام است از برنج‌های خریداری‌شده قسمتی به تهران حمل گردید
2,/mnt/mehrabikia/2TB/alldata/FarsSpon/FarsSpon_Retrying/225247404/225247404-1032-1.wav,دقیقا دنبال چی باید باشم
3,/mnt/mehrabikia/2TB/alldata/cv-corpus-6.1-2020-12-11/fa/clips/common_voice_fa_20323128.mp3,می توانید ریش مرا مرتب کنید، لطفا؟
4,/mnt/mehrabikia/2TB/alldata/FarsSpon/FarsSpon_Retrying/76597648/76597648-4090-1.wav,دلم براتون تنگ شده
5,/mnt/mehrabikia/2TB/alldata/FarsSpon/FarsSpon_Retrying/135136243/135136243-5340-1.wav,هیچ‌وقت قرار نبوده انجام بده
6,/mnt/mehrabikia/2TB/alldata/cv-corpus-6.1-2020-12-11/fa/clips/common_voice_fa_19335333.mp3,آیا فکر می کنید این جدی است؟
7,/mnt/mehrabikia/2TB/alldata/cv-corpus-6.1-2020-12-11/fa/clips/common_voice_fa_18615791.mp3,آدمبزرگی که رو تاب نشسته و پاهاش ثابت رو زمینه؛ یه لیپتون تموم شدهست
8,/mnt/mehrabikia/2TB/alldata/FarsSpon/FarsSpon_Retrying/329500534/329500534-6030-1.wav,انگار بیشتر از بیست درصد تخفیف داشته
9,/mnt/mehrabikia/2TB/alldata/FarsSpon/FarsSpon_Retrying/101715077/101715077-3145-1.wav,من فقط برای خدافظی اومدم اینجا


In [ ]:
dictionary_mapping = {
    'ك': 'ک', 'دِ': 'د', 'بِ': 'ب', 'زِ': 'ز', 'ذِ': 'ذ', 'شِ': 'ش', 'سِ': 'س', 'ى': 'ی',
    'ي': 'ی', 'أ': 'ا', 'ؤ': 'و', "ے": "ی", "ۀ": "ه", "ﭘ": "پ", "ﮐ": "ک", "ﯽ": "ی",
    "ﺎ": "ا", "ﺑ": "ب", "ﺘ": "ت", "ﺧ": "خ", "ﺩ": "د", "ﺱ": "س", "ﻀ": "ض", "ﻌ": "ع",
    "ﻟ": "ل", "ﻡ": "م", "ﻢ": "م", "ﻪ": "ه", "ﻮ": "و", 'ﺍ': "ا", 'ة': "ه",
    'ﯾ': "ی", 'ﯿ': "ی", 'ﺒ': "ب", 'ﺖ': "ت", 'ﺪ': "د", 'ﺮ': "ر", 'ﺴ': "س", 'ﺷ': "ش",
    'ﺸ': "ش", 'ﻋ': "ع", 'ﻤ': "م", 'ﻥ': "ن", 'ﻧ': "ن", 'ﻭ': "و", 'ﺭ': "ر", "ﮔ": "گ",

    "a": "‌ای‌", "b": "‌بی‌", "c": "‌سی‌", "d": "‌دی‌", "e": "‌ایی‌", "f": "‌اف‌",
    "g": "‌جی‌", "h": "‌اچ‌", "i": "‌آی‌", "j": "‌جی‌", "k": "‌کی‌", "l": "‌ال‌",
    "m": "‌ام‌", "n": "‌ان‌", "o": "‌او‌", "p": "‌پی‌", "q": "‌کیو‌", "r": "‌آر‌",
    "s": "‌اس‌", "t": "‌تی‌", "u": "‌یو‌", "v": "‌وی‌", "w": "‌دبلیو‌", "x": "‌اکس‌",
    "y": "‌وای‌", "z": "‌زد ",
    "\u200c": " ", "\u200d": " ", "\u200e": " ", "\u200f": " ", "\ufeff": " ",

    "نو آوری‌مان": "نو‌آوری‌مان",
    "نو آوری مان": "نو‌آوری‌مان",
    "نو آوریمان": "نو‌آوری‌مان",
    " ا م ": "‌ام ",
    " م ": "‌ام ",
    "کنندهای": "کننده‌ای",
    "ارائهای": "ارائه‌ای",
    "ایدهای": "ایده‌ای",
    "ماسهای": "ماسه‌ای",
    "خامنهای": "خامنه‌ای",
    "قلهای": "قله‌ای",
    "سیارهای": "سیاره‌ای",
    "کیسهای": "کیسه‌ای",
    "شانهای": "شانه‌ای",
    "غریبهای": "غریبه‌ای",
    "برنامهای": "برنامه‌ای",
    "سختگیرانهای": "سختگیرانه‌ای",
    "بهانهای": "بهانه‌ای",
    "زیرروالهای": "زیر روالهای",
    "درهای": "دره‌ای",
    "آمادهای": "آماده‌ای",
    "سادهای": "ساده‌ای",
    "سرمایهگذارهای": "سرمایه گذارهای",
    "فوقالعادهای": "فوق‌العاده‌ای",
    "حادثهای": "حادثه‌ای",
    "نویسندههای": "نویسنده‌های",
    "علاقهای": "علاقه‌ای",
    "برجستهای": "برجسته‌ای",
    "جلگهای": "جلگه‌ای",
    "زندهای": "زنده‌ای",
    "فنآوریهای": "فناوری‌های",
    "سایهروشنهای": "سایه روشن‌های",
    "بیسابقهای": "بی سابقه‌ای",
    "فرضیهای": "فرضیه‌ای",
    "راهاندازهای": "راه اندازهای",
    "بیشهای": "بیشه‌ای",
    "مقالهای": "مقاله‌ای",
    "دیگهای": "دیگه‌ای",
    "ماههاست": "ماه هاست",
    "نرمافزارهای": "نرم‌افزارهای",
    "کتابسوزانهای": "کتاب سوزان‌های",
    "سیستمعاملهای": "سیستم عامل‌های",
    "اسلحهای": "اسلحه‌ای",
    "وقفهای": "وقفه‌ای",
    "زمینهای": "زمینه‌ای",
    "حرامزادههای": "حرامزاده‌های",
    "هزینهای": "هزینه‌ای",
    "انداختهای": "انداخته‌ای",
    "جسورانهای": "جسورانه‌ای",
    "فاجعهای": "فاجعه‌ای",
    "جامعهای": "جامعه‌ای",
    "پدیدهای": "پدیده‌ای",
    "اغواگرانهای": "اغواگرانه‌ای",
    "تکانهای": "تکانه‌ای",
    "لولهای": "لوله‌ای",
    "نشانهای": "نشانه‌ای",
    "وسیلهای": "وسیله‌ای",
    "آیندهای": "آینده‌ای",
    "بردهای": "برده‌ای",
    "سابقهای": "سابقه‌ای",
    "ناحیهای": "ناحیه‌ای",
    "تکاندهندهای": "تکان دهنده‌ای",
    "بودجهای": "بودجه‌ای",
    "روزانهای": "روزانه‌ای",
    "چارهای": "چاره‌ای",
    "انگیزهای": "انگیزه‌ای",
    "دادهای": "داده‌ای",
    "عدهای": "عده‌ای",
    "هفتهای": "هفته‌ای",
    "منطقهای": "منطقه‌ای",
    "استارتآپهای": "استارتاپ‌های",
    "سازهای": "سازه‌ای",
    "مجموعهای": "مجموعه‌ای",
    "فلسفهای": "فلسفه‌ای",
    "تذکردهندهای": "تذکر دهنده‌ای",
    "مصاحبهای": "مصابحه‌ای",
    "نمونهای": "نمونه‌ای",
    "قلمموهای": "قلم مو‌های",
    "شبزندهداری": "شب زنده‌داری",
    "خوردهباشد": "خورده باشد",
    "داشتهباشید": "داشته باشید",
    "فزایندهای": "فزاینده‌ای",
    "عمدهای": "عمده‌ای",
    "بدیهایی": "بدی‌های",
    "نوشت‌هایم": "نوشته‌ایم",
    "بنتالهدی": "بنت الهدی",
    "نوشتهام": "نوشته‌ام",
    "سرمایهگذاران": "سرمایه گذاران",
    "خانهی": "خانه‌ی",
    "گستاخانهی": "گستاخانه‌ی",
    "گرفتهباشیم": "گرفته باشیم",
    "خونهی": "خونه‌ی",
    "داشتهام": "داشته‌ام",
    "رشتهام": "رشته‌ام",
    "سرمایهگذارانشان": "سرمایه گذارانشان",
    "ریشهکنی": "ریشه‌کنی",
    "مودبانهتری": "مودبانه‌تری",
    "برگردانشدهاند": "برگردان شده‌اند",
    "قرمهسبزی": "قرمه‌سبزی",
    "راهجویی": "راه جویی",
    "اماهیچوقت": "اما هیچوقت",
    "آبوهوای": "آب و هوای",
    "بقیهاش": "بقیه‌اش",
    "طبقهبندی": "طبقه‌بندی",
    "مردههان": "مرده هان",
    "آمادهاند": "آماده‌اند",
    "نشدهاید": "نشده‌اید",
    "آگاهیرسانی": "آگاهی رسانی",
    "نداشتهاند": "نداشته‌اند",
    "شکنانهترین": "شکنانه‌ترین",
    "اقدامهایی": "اقدام‌هایی",
    "راهآهن": "راه آهن",
    "شدهاند": "شده‌اند",
    "تازهترین": "تازه‌ترین",
    "روبهروی": "رو به روی",
    "منحصربهفرد": "منحصر به فرد",
    "سیزدهبدر": "سیزده بدر",
    "برندهی": "برنده‌ی",
    "خانهاشتراکی": "خانه اشتراکی",
    "دادههایی": "داده‌هایی",
    "استفادهتر": "استفاده‌تر",
    "گذرنامهتان": "گذرنامه‌تان",
    "کهنترین": "کهنه‌ترین",
    "فرهنگسرا": "فرهنگ‌سرا",
    "آمادهاید": "آماده‌اید",
    "ویژهی": "ویژه‌ی",
    "غریزهات": "غریزه‌ات",
    "مادرشوهری": "مادر شوهری",
    "نبودهام": "نبوده‌ام",
    "بودهاند": "بوده‌اند",
    "وتنها": "و تنها",
    "بداههکاری": "بداهه‌کاری",
    "سرمایهگذار": "سرمایه گذار",
    "برنامهنویس": "برنامه نویس",
    "مهنازخانم": "مهناز خانم",
    "مواجهاند": "مواجه‌اند",
    "توسعهاش": "توسعه‌اش",
    "سینهام": "سینه‌ام",
    "سین‌هام": "سینه‌ام",
    "نمیخواهند": "نمیخواهند",
    "فنآوری‌ها": "فناوری‌ها",
    "دنبالهرو": "دنباله‌رو",
    "لبهی": "لبه‌ی",
    "اللهیار": "الله یار",
    "ارزندهتر": "ارزنده‌تر",
    "برههای": "بره‌ای",
    "پیادهسازی": "پیاده‌سازی",
    "دهسالگی": "ده سالگی",
    "رسانهای": "رسانه‌ای",
    "ریشسفیدها": "ریش سفید‌ها",
    "چهجوری": "چه جوری",
    "ویژگیهایی": "ویژگی‌هایی",
    "می‌فهمی‌م": "میفهمیم",
    "وبهم": "و بهم",
    "قطرهای": "قطره‌ای",
    "ازتنهایی": "از تنهایی",
    "لطیفهای": "لطیفه‌ای",
    "باشهاومدم": "باشه اومدم",
    "منحصربهفردترین": "منحصر به فرد‌ترین",
    "کردهاند": "کرده‌اند",
    "اندازهای": "اندازه‌ای",
    "بهرهبرداری": "بهره برداری",
    "اماشوهرجان": "اما شوهر جان",
    "خانوادهاش": "خانواده‌اش",
    "نشدهاند": "نشده‌اند",
    "نکردهایم": "نکرده‌ایم",
    "تخممرغ‌هایش": "تخم مرغ‌هایش",
    "وظیفهش": "وظیفه‌اش",
    "مشگینشهر": "مشگی شهر",
    "توسعهدهندگانش": "توسعه دهندگانش",
    "امینابراهیم": "امین ابراهیم",
    "دربارهاش": "درباره‌اش",
    "میانافزارها": "میان‌افزارها",
    "دیدهاند": "دیده‌اند",
    "خانوادهام": "خانواده‌ام",
    "مایهی": "مایه‌ی",
    "نوشتهشدن": "نوشته شدن",
    "راهحل‌هایشان": "راه حل‌هایشان",
    "میهماننواز": "میهمان نواز",
    "زیبندهی": "زیرنده‌ی",
    "راههایی": "راه‌هایی",
    "جربزهی": "جربزه‌ی",
    "بهجا": " به جا",
    "بطورهمزمان": "به طور همزمان",
    "فهمیدهبود": "فهمیده بود",
    "دوربرگردان‌ها": "دور برگردان‌ها",
    "شالودهی": "شالوده‌ی",
    "راهکاریی": "راه‌کاری",
    "مخالفتهایی": "مخالفت‌هایی",
    "چیزهاازشون": "چیزها ازشون",
    "سکونتگاه‌های": "سکونت گاه‌های",
    "سالهابود": "سال‌ها بود",
    "نمونهی": "نمونه‌ی",
    "سرمایهگذاری": "سرمایه گذاری",
    "شبکهای": "شبکه‌ای",
    "خواهرشوهر": "خواهر شوهر",
    "سرگیجهآور": "سرگیجه آور",
    "آستانهی": "آستانه‌ی",
    "دادهاست": "داده است",
    "مجسمهسازی": "مجسمه سازی",
    "ماهرانهترین": "ماهرانه‌ترین",
    "پنجشنبههایی": "پنجشنبه شب‌هایی",
    "نرفنهام": "نرفته‌ام",
    "قورمهسبزی": "قورمه سبزی",
    "گذارهای": "گذاره‌ای",
    "بندهخدا": "بنده خدا",
    "روزنامهنگاران": "روزنامه نگاران",
    "نقشهی": "نقشه‌ی",
    "حملهی": "حمله‌ی",
    "تکنیکهاست": "تکنیک هاست",
    "نرمافزارهایمان": "نرم‌افرارهایمان",
    "مادرشوهرم": "مادر شوهرم",
    "ماهگیمون": "ماه گیمون",
    "مادرشوهرمحترم": "مادر شوهر محترم",
    "شوهرداری": "شوهر داری",
    "سرمایهگذارها": "سرمایه گذارها",
    "بهرهمند": "بهره‌مند",
    "درمانهایی": "درمان‌هایی",
    "عامدانهتر": "عامدانه‌تر",
    "تازهوارد": "تازه وارد",
    "مونتهویدئو": "مونته ویدئو",
    "ذائق‌هاش": "ذائقه‌اش",
    "گوشهگیرتر": "گوشه‌گیرتر",
    "دنبالهدار": "دنباله‌دار",
    "بیخانمان‌ها": "بی‌خانمان‌ها",
    "سرمایهدارها": "سرمایه‌دارها",
    "مادرشوهریم": "مادر شوهریم",
    "صبحان‌هاش": "صبحانه‌اش",
    "جنازهست": "جنازه است",
    "شمارهات": "شماره‌ای",
    "بهقدری": "به قدری",
    "کیسهی": "کیسه‌ی",
    "کوششهایی": "کوشش‌هایی",
    "مادرشوهر": "مادر شوهر",
    "رابطهی": "رابطه‌ی",
    "نوشتهاند": "نوشته‌اند",
    "کنجکاوانهی": "کنجکاوانه‌ی",
    "غیرمتعهد": "غیر متعهد",
    "کردهای": "کرده‌ای",
    "وهمکارانم": "و همکارانم",
    "گردهمآیی": "گردهمایی",
    "اللهوردی": "الله وردی",
    "صرفهجویی": "صرفه جویی",
    "ماندهاند": "مانده‌اند",
    "برنامهنویسی": "برنامه‌نویسی",
    "امینمهدی": "امین مهدی",
    "سهامدارنی": "سهام دارانی",
    "مسابقهی": "مسابقه‌ی",
    "ستارهشناسم": "ستار شناسم",
    "گرفتهاند": "گرفته‌اند",
    "جامعهشان": "جامعه‌شان",
    "بچهی": "بچه‌ی",
    "شیوهی": "شیوه‌ی",
    "بهکار": "به کار",
    "بهتراست": "بهتر است",
    "سروکلهشون": "سر و کلهشون",
    "رسیدهمسرش": "رسید همسرش",
    "پسراهل": "پسر اهل",
    "پروژههای": "پروژه‌های",
    "عاقلان‌هام": "عاقلانه‌ام",
    "گذاشتهاند": "گذاشته‌اند",
    "کردهام": "کرده‌ام",
    "اندازهگیری": "اندازه گیری",
    "یاوهگویی": "یاوه گویی",
    "سازمانهایی": "سازمان‌هایی",
    "نمودهاند": "نموده‌اند",
    "تنهاییآور": "تنهایی آور",
    "قراردهیم": "قرار دهیم",
    "ازشوهرجان": "از شوهر جان",
    "کرهجنوبی": "کره جنوبی",
    "توهینآمیز": "توهین آمیز",
    "فنآوریهایی": "فناوری‌هایی",
    "داشتهاید": "داشته‌اید",
    "شدهایم": "شده‌ایم",
    "نمیفهمم": "نمیفهمم",
    "مثالهایی": "مثال‌هایی",
    "رییسجمهور": "رییس جمهور",
    "مجموعهی": "مجموعه‌ی",
    "درندهاند": "درنده‌اند",
    "امابهش": "اما بهش",
    "بازخواهند": "باز خواهند",
    "برنامههایی": "برنامه‌هایی",
    "یهجا": "یه جا",
    "زگیلهایی": "زگیل‌هایی",
    "وسیلهی": "وسیله‌ی",
    "بهمنیار": "بهمن یار",
    "دادهام": "داده‌ام",
    "بههنگام": "به هنگام",
    "بهدروغ": "به دروغ",
    "دورافتادهترین": "دور افتاده‌ترین",
    "نامهایی": "نامه‌ایی",
    "سهقسمتی": "سه قسمتی",
    "توجهازچیدن": "توجه از چیدن",
    "پیامرسان‌ها": "پیام رسان‌ها",
    "بهمنزاد": "بهمن زاد",
    "نشانههایی": "نشانه‌هایی",
    "راهحل‌های": "راه حل‌های",
    "راهحلهایی": "راه حل‌هایی",
    "راهحلهای": "راه حل‌های",
    "نظرخواهی‌ها": "نظر خواهی‌ها",
    "نظرخواهیها": "نظر خواهی‌ها",
    "کندهی": "کنده‌ی",
    "حرامزاده‌های": "حرام زاده‌های",
    "شبیهسازیهایی": "شبیه سازی‌هایی",
    "مهارتهایی": "مهارت‌هایی",
    "روبهرویشان": "رو به رویشان",
    "برجستهترین": "برجسته‌ترین",
    "نمیفهمیدم": "نمیفهمیدم",
    "دستگاههایی": "دستگاه‌هایی",
    "برادرشوهر": "برادر شوهر",
    "گرسن‌هام": "گرسته‌ام",
    "گرسنههام": "گرسته‌ام",
    "قهوهخوری": "قهوه خوری",
    "دادهاید": "داده‌اید",
    "بهآرامی": "به آرمانی",
    "دانستنیهاست": "دانستنی‌هاست",
    "بهراحتی": "به راحتی",
    "ایدهپردازی": "ایده‌پردازی",
    "ریشسفیدهای": "ریش سفید‌های",
    "خفهمون": "خفه مون",
    "بهجای": "به جای",
    "ریزخشونت‌ها": "ریز خشونت‌ها",
    "ریزخشونتها": "ریز خشونت‌ها",
    "حساسیتهایی": "حساسیت‌هایی",
    "پشتصحنهی": "پشت صحنه‌ی",
    "کلهی": "کله‌ی",
    "تاشوهرم": "تا شوهرم",
    "آیندهاش": "آینده‌اش",
    "پروانههایی": "پروانه‌هایی",
    "خوبیهایی": "خوبی‌هایی",
    "نرمافزارها": "نرم‌افزارها",
    "رساندهاند": "رسانده‌اند",
    "سرمایهگذارنی": "سرمایه گذارانی",
    "تکهچسبانی": "تکه چسبانی",
    "بیتوجهی": "بی توجهی",
    "جاهطلبی": "جاه طلبی",
    "پرغلغلهتان": "پر غلغله‌تان",
    "خمینیشهر": "خمینی شهر",
    "رشتهتوییت": "رشته توییت",
    "موهبتهایی": "موهبت‌هایی",
    "برنامهی": "برنامه‌ی",
    "مادرشوهردارم": "مادر شوهر داردم",
    "سیاهپوستان": "سیاه پوستان",
    "شرکتهایی": "شرکت‌هایی",
    "نیاوردهاند": "نیاورده‌اند",
    "آنهم": "آن هم",
    "شوهرداریم": "شوهر داریم",
    "یکچهارم": "یک چهارم",
    "پروندههاست": "پرونده هاست",
    "برنامهت": "برنامه‌ات",
    "چروکیدهمان": "چروکیده‌مان",
    "زمینهسازی": "زمینه سازی",
    "زدهاند": "زده‌اند",
    "اظهارنظرپرداختن": "اظهار نظر پرداختن",
    "صلحطلبانهترین": "صلح طلبانه‌ترین",
    "بهغلط": "به غلط",
    "ایدهآلم": "ایده آلم",
    "سیاهکاران": "سیاه کاران",
    "امیرابراهیم": "امیر ابراهیم",
    "توسعهدهندگان": "توسعه دهندگان",
    "لحظهی": "لحظه‌ی",
    "امینطاها": "امین طاها",
    "بینالنهرین": "بین النهرین",
    "نیمهوقت": "نیمه وقت",
    "پیادهروی": "پیاده روی",
    "آلودهاند": "آلوده‌اند",
    "گریهکرد": "گره کرد",
    "نعمتهایی": "نعمت‌هایی",
    "مادرشوهرشماهم": "مادر شوهر شما هم",
    "آشپزخونهاس": "آشپزخونه‌اس",
    "مسابقهها": "مسابقه‌ها",
    "مسابقهای": "مسابقه‌های",
    "برنامهریزی": "برنامه‌ریزی",
    "بازخواهید": "باز خواهید",
    "جوییما": "جویی ما",
    "آماده ایم": "آماده‌ایم",
    "مدلسازی": "مدل‌سازی",
    "درصورتیکه": "در صورتیکه",
    "آمریکاییات": "آمریکایی‌ات",
    "مادریاش": "مادری‌اش",
    "غافلگیرکننده": "غافلگیر کننده",
    "پیکرتراشی": "پیکر تراشی",
    "اذیتوآزار": "اذیت و آزار",
    "امتیازاورترین": "امتیاز آور",
    "جیکجیک": "جیک جیک",
    "تاشب": "تا شب",
    "کپیرایت": "کپی رایت",
    "آنتیبادی": "آنتی بادی",
    "عجیبتر": "عجیب‌تر",
    "استانداردسازی": "استاندارد سازی",
    "هشتادوهشت": "هشتاد و هشت",
    "متنوعتر": "متنوع‌تر",
    "منظورانجام": "منظور انجام",
    "نگرانکننده‌ترین": "نگران کننده‌ترین",
    "شگفتانگیز": "شگفت انگیز",
    "رنگینپوست": "رنگین پوست",
    "فارغ التحصیلان": "فارغ‌التحصیلان",
    "ترسناکتر": "ترسناک‌تر",
    "لا رامبلا": "لارامبلا",
    "پرجمعیتترین": "پرجمعیت‌ترین",
    "درمیآیند": "درمی‌آیند",
    "باشمالکی": "باشم الکی",
    "وسیعتر": "وسیع‌تر",
    "فاحشهخانه": "فاحشه خانه",
    "بااحتیاط": "با احتیاط",
    "قانعکننده": "قانع‌کننده",
    "انعطافپذیری": "انعطاف‌پذیری",
    "بیتالمقدس": "بیت‌المقدس",
    "اوپناستریتمپ": "اوپن استریت مپ",
    "روزابارونی": "روزا بارونی",
    "محافظهکارانه": "محافظه کارانه",
    "فوتبالدستی": "فوتبال دستی",
    "توسعهدهنده": "توسعه دهنده",
    "قانونگزاران": "قانون گزاران",
    "العسریسرا": "العسر یسرا",
    "خارقالعاده": "خارق‌العاده",
    "بیماریمزمن": "بیماری مزمن",
    "بادوستانتان": "با دوستانتان",
    "برابربیشتر": "برابر بیشتر",
    "ارائهدهنده": "ارائه دهنده",
    "طوفانزدگان": "طوفان زندگان",
    "امینمحمد": "امین محمد",
    "محیطزیست": "محیط زیست",
    "شقیترینشان": "شقی‌ترینشان",
    "بودواقعا": "بود واقعا",
    "نیویورکتایمز": "نیویورک تایمز",
    "ریودوژانیرو": "ریو دو ژانیرو",
    "مشترکالمنافع": "مشترک‌المنافع",
    "اسلایدسازم": "اسلاید سازم",
    "نمیآوریدش": "نمی‌آوریدش",
    "بینالملل": "بین‌الملل",
    "مصرفکنندگان": "مصرف کنندگان",
    "امینالدین": "امین الدین",
    "امریکااینقدر": "امریکا اینقدر",
    "بعضیاوقات": "بعضی اوقات",
    "خاطربچه": "خاطر بچه",
    "ایناکیلویی": "اینا کیلویی",
    "ویکیپدیا": "ویکی‌پدیا",
    "مافکرمیکنیم": "ما فکر میکنیم",
    "انگلیسیزبان": "انگلیسی زبان",
    "کلهشون": "کله‌شون",
    "آدمبزرگی": "آرم بزرگی",
    "مر آت مر آه": "مر‌آت مر‌آت",
    "آسیبزد": "آسیب زد",
    "آیآرسی": "آی آرسی",
    "آسیااقیانوسیه": "آسیا اقیانوسیه",
    "آیای": "آیا",
    "میانجنسی": "میان جنسی",
    "میاننسلی": "میان نسلی",
    "میان‌افزار‌ها": "میان افزارها",
    "آییننامه": "آیین‌نامه",
    "ارائهشده": "ارائه‌شده",
    "اشپزخونه": "آشپزخونه",
    "اماعلتشونمیپرسه": "اما علتشو نمیپرسه",
    "امیدوارکننده": "امیدوار کننده",
    "ایالاتمتحده": "ایالات متحده",
    "بااینکه": "با اینکه",
    "بلندپروازانه": "بلند پروازانه",
    "بهترازاینه": "بهتر از اینه",
    "بهدست‌آمده": "به دست‌آمده",
    "بهوسیله": "به وسیله",
    "بیادبانه": "بی ادبانه",
    "بیاندازه": "بی اندازه",
    "بیصبرانه": "بی صبرانه",
    "بیفایده": "بی فایده",
    "بیمهره": "بی مهره",
    "بینظیره": "بی نظیره",
    "تاریخزده": "تاریخ زده",
    "تهرانزده": "تهران زده",
    "تولیدشده": "تولید شده",
    "تولیدکننده": "تولید کننده",
    "تکمیلشده": "تکمیل شده",
    "جاافتاده": "جا افتاده",
    "جمع‌آوریکننده": "جمع‌ آوری کننده",
    "جورآدمیه": "جور آدمیه",
    "حقالزحمه": "حق الزحمه",
    "دخترونهتره": "دخترونه تره",
    "دوپنجره": "دو پنجره",
    "ذاتالریه": "ذات‌الریه",
    "راسالخیمه": "راس‌الخیمه",
    "رنگماده": "رنگ ماده",
    "سوئاستفاده": "سو استفاده",
    "سواستفاده": "سو استفاده",
    "شبهجزیره": "شبه جزیره",
    "صادرکننده": "صادر کننده",
    "ضررداره": "ضرر داره",
    "عابرپیاده": "عابر پیاده",
    "فوقالعاده": "فوق‌العاده",
    "قابلتوجه": "قابل توجه",
    "قانع‌کننده": "قانع‌ کننده",
    "مادربیچاره": "مادر بیچاره",
    "مشخصشده": "مشخص شده",
    "مصرفکننده": "مصرف کننده",
    "مصیبتزده": "مصیب تزده",
    "ناامیدکننده": "ناامید کننده",
    "نیمفاصله": "نیم‌فاصله",
    "هماهنگکننده": "هماهنگ کننده",
    "همهجانبه": "همه جانبه",
    "واردکننده": "وارد کننده",
    "وخوابگاه": "و خوابگاه",
    "ودستگاه": "و دستگاه",
    "وزردچوبه": "و زردچوبه",
    "وپروانه": "و پروانه",
    "پدرخوانده": "پدر خوانده",
    "چاپشده": "چاپ شده",
    "کردته": "کرد ته",
    "کردندکه": "کردند که",
    "یکطرفه": "یک طرفه",
    "پایینتره": "پایین‌تره",
    "اشتراکگذاری": "اشتراک گذاری",
    "انحصارگراناند": "انحصار گران‌اند",
    "خوشحالییییی": "خوشحالی",
    "همتیمی‌هایشان": "هم تیمی‌هایشان",
    "پایدار‌ام‌باید": "پایدار‌ام ‌باید",
    "پرجنبوجوش‌تر": "پر جنب و جوش‌تر",
    "آبمروارید": "آب مروارید",
    "آتشسوزی": "آتش سوزی",
    "آتشنشانی": "آتش‌نشانی",
    "آتشنشان": "آتش‌نشان",
    "آرامشبخش": "آرامش بخش",
    "آشناداشتن": "آشنا داشتن",
    "آقاچیزی": "آقا چیزی",
    "آموخت‌هام": "آموخته‌ام",
    "آموزششان": "آموزش‌شان",
    "ازآنجا": "از آنجا",
    "ازالان": "از الان",
    "ازاینجا": "از اینجا",
    "ازجیبش": "از جیبش",
    "ازدستش": "از دستش",
    "ازدیوار": "از دیوار",
    "ازشغلشون": "از شغلشون",
    "ازوقتی": "از وقتی",
    "ازکسانی": "از کسانی",
    "اسباببازی": "اسباب بازی",
    "اسبسوار": "اسب سوار",
    "اصیلزاده": "اصیل زاده",
    "افتادهاید": "افتاده‌اید",
    "ال‌هام": "الهام",
    "امااصلا": "اما اصلا",
    "امااصلابه": "اما اصلا به",
    "امااین": "اما این",
    "امابعد": "اما بعد",
    "امابعدیکی": "اما بعد یکی",
    "اماجاذبه": "اما جاذبه",
    "امرارمعاش": "امرار معاش",
    "امکانپذیر": "امکان پذیر",
    "انت‌های": "انتهای",
    "انت‌هایی": "انتهایی",
    "ایزدبانوی": "ایزد بانوی",
    "بااینحال": "با اینحال",
    "باحتمال": "به احتمال",
    "باحجاب": "با حجاب",
    "باخنده": "با خنده",
    "بادوستاش": "با دوستاش",
    "بارمان": "بار مان",
    "باز‌تر": "باز ‌تر",
    "باطعنه": "با طعنه",
    "بافریاد": "با فریاد",
    "بارگزاری": "بارگذاری",
    "بالامنم": "بالا منم",
    "بگیرمامان": "بگیر مامان",
    "بیاحترامی": "بی احترامی",
    "بیادبی": "بی ادبی",
    "بیاعتنا": "بی اعتنا",
    "بیدارباش": "بیدار باش",
    "بیشازحد": "بیش از حد",
    "بیمسئولیت": "بی مسئولیت",
    "تاسفبار": "تاسف بار",
    "تامشکلمون": "تا مشکلمون",
    "تانقشه": "تا نقشه",
    "تصمیمگیری": "تصمیم گیری",
    "تقسیمبندی": "تقسیم بندی",
    "تقصیرارو": "تقصیرا رو",
    "جدیدابرای": "جدیدا برای",
    "جعبهابزار": "جعبه ابزار",
    "جلوتونو": "جلو تو نو",
    "حاضردر": "حاضر در",
    "حاضرنیست": "حاضر نیست",
    "دستنخورده": "دست نخورده",
    "دوامتیاز": "دو امتیاز",
    "دوروزتمام": "دو روز تمام",
    "شخصیسازی": "شخصی‌سازی",
    "شدواجناس": "شد و اجناس",
    "شوهردارم": "شوهر دارم",
    "شوهرشماهم": "شوهر شما هم",
    "شوهرمحترم": "شوهر محترم",
    "شکلگیری": "شکل گیری",
    "صخرهنوردی": "صخره‌نوردی",
    "صدوبیست": "صد و بیست",
    "عقبنشینی": "عقب نشینی",
    "عکسالعمل": "عکس‌العمل",
    "غرغرمیکنم": "غرغر میکنم",
    "هزاربار": "هزار بار",
    "هزارتومان": "هزار تومان",
    "هزارجور": "هزار جور",
    "هزاروسیصد": "هزار و سیصد",
    "هممیهنان": "هم میهنان",
    "هممیهنانش": "هم میهنانش",
    "همنسلانش": "هم نسلانش",
    "همهگیری": "همه گیری",
    "هییییچ": "هیچ",
    "وقتاخیلی": "وقتا خیلی",
    "وقتابه": "وقتا به",
    "وقتگذرانی": "وقت گذرانی",
    "ومحکوم": "و محکوم",
    "ومحیط‌ها": "و محیط‌ها",
    "وکشورتان": "و کشورتان",
    "ویکیمدیا": "ویکی‌مدیا",
    "یهوگفت": "یهو گفت",
    "اینجااز": "اینجا از",
}

fixator_dictionary = {
    "ب‌های": "بهای",
    "به‌ترین": "بهترین",
    "آس‌تر": "‌آستر",
    "ارکس‌تر": "ارکستر",
    "ان‌تر": "انتر",
    "بس‌تر": "بستر",
    "به‌تر": "بهتر",
    "به‌ترتر": "بهترتر",
    "توئی‌تر": "تویتتر",
    "تویی‌تر": "توییتر",
    "تی‌تر": "تیتر",
    "دخ‌تر": "دختر",
    "دف‌تر": "دفتر",
    "دلس‌تر": "دلستر",
    "دک‌تر": "دکتر",
    "ش‌تر": "شتر",
    "لی‌تر": "لیتر",
    "م‌تر": "متر",
    "هیپس‌تر": "هیپستر",
    "پی‌تر": "پیتر",
    "چ‌تر": "چتر",
    "کم‌تر": "کمتر",
    "گنگس‌تر": "گنگستر",
    "انگش‌تر": "انگشتر",
    "سن‌تر": "سنتر",
    "تویت‌تر": "توییتر",
    "مادهش‌تر": "ماده شتر",
    "وی‌ترین": "ویترین",
    "کرونوم‌تر": "کرنومتر",
    "که‌تر": "کهتر",
    "فیل‌تر": "فیلتر",
    "ال‌هام": "الهام",
    "آل‌مان": "آلمان",
    "انت‌های": "انتهای",
    "انت‌هایی": "انتهایی",
    "آموخت‌هام": "آموخته‌ام",
}

In [ ]:
from parsivar import Normalizer

import num2fawords
import re
import string


_normalizer = Normalizer(half_space_char="\u200c", statistical_space_correction=True)
chars_to_ignore = [
    ",", "?", ".", "!", "-", ";", ":", '""', "%", "'", '"', "�",
    "#", "!", "؟", "?", "«", "»", "،", "(", ")", "؛", "'ٔ", "٬", 'ٔ', ",", "?",
    ".", "!", "-", ";", ":", '"', "“", "%", "‘", "”", "�", "–", "…", "_", "”", '“', '„',
    'ā', 'š', 'ّ', 'ْ',
]

chars_to_ignore = chars_to_ignore + list(string.ascii_lowercase + string.digits)
chars_to_ignore = f"""[{"".join(chars_to_ignore)}]"""
zwnj = "\u200c"
silent_chars = ["ا", "د", "ذ", "ر", "ز", "و", "آ"] + [zwnj] + [" "]


def multiple_replace(text, chars_to_mapping):
    pattern = "|".join(map(re.escape, chars_to_mapping.keys()))
    return re.sub(pattern, lambda m: chars_to_mapping[m.group()], str(text))


def remove_special_characters(text, chars_to_ignore_regex):
    text = re.sub(chars_to_ignore_regex, '', text).lower() + " "
    return text


def convert_word_nums_to_text(word):
    try:
        word = int(word)
        word = num2fawords.words(word)
    except:
        word = word

    return word


def normalizer_at_word_level(text):
    words = text.split()
    _text = []

    for word in words:
        word = convert_word_nums_to_text(word)
        word = fixator_dictionary.get(word, word)

        _text.append(word)

    return " ".join(_text) + " "


def finder(ss, s, starter=False):
    found = []
    for m in re.finditer(ss, s):
        if starter:
            found.append(m.start())
        else:
            found.append((m.start(), m.end()))

    return found


def substring_replace(ss, s, start, end, stripped=True):
    s_start = s[:start]
    s_end = s[end:]

    counter = 0
    if stripped:
        counter = 1 if s_start.endswith(" ") else counter
        s_start = s_start.rstrip()

    return s_start + ss + s_end, counter


def normalizer(
        batch,
        is_normalize=True,
        return_dict=True,
        filter_trivials=False,
        remove_extra_space=False
):
    text = batch["sentence"].lower().strip()

    # Parsivar normalizer
    if is_normalize:
        text = _normalizer.normalize(text)

    # Dictionary mapping
    text = multiple_replace(text, dictionary_mapping)
    text = re.sub(" +", " ", text)

    # Remove specials
    text = remove_special_characters(text, chars_to_ignore)
    text = re.sub(" +", " ", text)

    # Replace connected آ
    special, pointer = "آ", int("0")
    for f in sorted(finder(special, text, True)):
        index = f + pointer - 1
        if len(text) >= index:
            if text[index] not in silent_chars:
                new_text, extra_pointer = substring_replace(
                    f"{text[index]}{zwnj}", text, index, index + 1, stripped=True)
                text = new_text
                pointer += 1 + 1 - 1 - extra_pointer

    # Replace connected ها
    pointer = int("0")
    special_list = [
        # "ام", "ای", "است", "ایم", "اید", "اند",
        "هایمان", "هایم", "هایت", "هایش",
        "هایتان", "هایشان", "هام", "هات",
        "هاتان", "هامون", "هامان", "هاش",
        "هاتون", "هاشان", "هاشون",
        "هایی", "های", "هاس", "ها"
    ]
    for special in special_list:
        pointer = 0
        text = text
        for f in sorted(finder(special, text, False)):
            start, end = f[0] + pointer - 1, f[1] + pointer - 1
            if len(text) >= (end + 1):
                if len(text) == (end + 1):
                    new_text, extra_pointer = substring_replace(
                        f"{zwnj}{special}",
                        text,
                        start + 1,
                        end + 1,
                        stripped=True)
                    text = new_text
                    pointer += 1 + 1 - 1 - extra_pointer
                else:
                    if text[end + 1] == " ":
                        new_text, extra_pointer = substring_replace(
                            f"{zwnj}{special}",
                            text,
                            start + 1,
                            end + 1,
                            stripped=True)
                        text = new_text
                        pointer += 1 + 1 - 1 - extra_pointer

    special, pointer = "افزار", int("0")
    for f in sorted(finder(special, text, False)):
        start, end = f[0] + pointer - 1, f[1] + pointer - 1

        if len(text) >= (end + 1):
            new_text, extra_pointer = substring_replace(f"{zwnj}{special}", text, start + 1, end + 1, stripped=True)
            text = new_text
            pointer += 1 + 1 - 1 - extra_pointer

    # Replace connected ها
    pointer = int("0")
    special_list = [
        "ترین", "تر"
    ]
    for special in special_list:
        pointer = 0
        text = text
        for f in sorted(finder(special, text, False)):
            start, end = f[0] + pointer - 1, f[1] + pointer - 1
            if len(text) >= (end + 1):
                if len(text) == (end + 1):
                    new_text, extra_pointer = substring_replace(
                        f"{zwnj}{special}",
                        text,
                        start + 1,
                        end + 1,
                        stripped=True)
                    text = new_text
                    pointer += 1 + 1 - 1 - extra_pointer
                else:
                    if text[end + 1] == " ":
                        new_text, extra_pointer = substring_replace(
                            f"{zwnj}{special}",
                            text,
                            start + 1,
                            end + 1,
                            stripped=True)
                        text = new_text
                        pointer += 1 + 1 - 1 - extra_pointer

    # Normalizer at word level
    text = normalizer_at_word_level(text)
    text = re.sub(" +", " ", text)

    if remove_extra_space:
        text = text.strip()
    else:
        text = text.strip() + " "

    if filter_trivials:
        if not len(text) > 2:
            text = None

    if not return_dict:
        return text

    batch["sentence"] = text
    return batch

In [ ]:
#Cleaning

# normalizer import normalizer

def cleaning(text):
    if not isinstance(text, str):
        return None

    return normalizer({"sentence": text}, return_dict=False)

##################################################################
test = pd.read_csv('/home/sadeghi/test_data_finall.csv', sep=",")
print(f"Step 0: {len(test)}")

test["status"] = test["path"].apply(lambda path: True if os.path.exists(path) else None)
test = test.dropna(subset=["path"])
test = test.drop("status", 1)
print(f"Step 1: {len(test)}")

test["sentence"] = test["sentence"].apply(lambda t: cleaning(t))
test = test.dropna(subset=["sentence"])
print(f"Step 2: {len(test)}")

test = test.reset_index(drop=True)
print(test.head())

test = test[["path", "sentence"]]
test.to_csv("/home/sadeghi/test_finall_cleaned.csv", sep=",", encoding="utf-8", index=False)
########################################################################


Step 0: 9230
Step 1: 9230


/tmp/ipykernel_21846/3447816596.py:21: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  test = test.drop("status", 1)


Step 2: 9230
                                                path  \
0  /mnt/mehrabikia/2TB/alldata/FarsSpon/FarsSpon_...   
1  /mnt/mehrabikia/2TB/alldata/DeepMine/wav/trans...   
2  /mnt/mehrabikia/2TB/alldata/FarsSpon/FarsSpon_...   
3  /mnt/mehrabikia/2TB/alldata/FarsSpon/FarsSpon_...   
4  /mnt/mehrabikia/2TB/alldata/DeepMine/wav/trans...   

                                            sentence  
0                                               چیه   
1  در بخش دوم ضمن آشنایی با خواص گاز‌ها پیرامون ه...  
2                      می دونستی که این اتفاق میفته   
3                      چه کار میخوایی بکنی منو بکشی   
4  کوریا سفید گونه ای گیاه بوته ای شکل است که از ...  


In [ ]:
train = pd.read_csv('/home/sadeghi/train_data_finall.csv', sep=",")
print(f"Step 0: {len(train)}")

train["status"] = train["path"].apply(lambda path: True if os.path.exists(path) else None)
train = train.dropna(subset=["path"])
train = train.drop("status", 1)
print(f"Step 1: {len(train)}")

train["sentence"] = train["sentence"].apply(lambda t: cleaning(t))
train = train.dropna(subset=["sentence"])
print(f"Step 2: {len(train)}")

train = train.reset_index(drop=True)
print(train.head())

train = train[["path", "sentence"]]
train.to_csv("/home/sadeghi/train_finall_cleaned.csv", sep=",", encoding="utf-8", index=False)

Step 0: 913840


/tmp/ipykernel_21846/3299828736.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  train = train.drop("status", 1)


Step 1: 913840
Step 2: 913839
                                                path  \
0  /mnt/mehrabikia/2TB/alldata/FarsSpon/FarsSpon_...   
1  /mnt/mehrabikia/2TB/alldata/cv-corpus-6.1-2020...   
2  /mnt/mehrabikia/2TB/alldata/cv-corpus-6.1-2020...   
3  /mnt/mehrabikia/2TB/alldata/cv-corpus-6.1-2020...   
4  /mnt/mehrabikia/2TB/alldata/FarsSpon/FarsSpon_...   

                               sentence  
0           همشون تروریست به حساب میان   
1                من قبلا اینطوری نبودم   
2             از آشنایی با شما خوشبختم   
3                چطور فهمیدی من اینجام   
4  تو باید همین ال‌آن اینجا رو ترک کنی   
